In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import os
import pix2pix
import cv2
import matplotlib.pyplot as plt
from pix2pix import models
import tqdm
from torch2trt import torch2trt
from pix2pix import config

In [3]:
gt_path = 'videos/input_video_gt.mp4'
noisy_path = 'videos/input_video_noisy.mp4'
pred_save_path = 'videos/predicted_video_v4.mp4'

device = "cuda" if torch.cuda.is_available() else "cpu"
bs = 256
print (device)

cuda


In [4]:
model = models.unet(norm_layer=torch.nn.InstanceNorm2d, **config.model.GEN_ARGS)
model = torch.nn.DataParallel(model)
# model.load_state_dict(torch.load('../../models/pix2pix_64_11112021_115558/checkpoints/gen_epoch_2_pix2pix_64_11112021_115558.pt'))
model.load_state_dict(torch.load('../../models/pix2pix_32_13112021_183033/checkpoints/gen_epoch_6_pix2pix_32_13112021_183033.pt'))
model = model.module
model.eval();
model_trt = torch2trt(model, [torch.ones(1, 3, 256, 256).to(device)], max_batch_size=bs)

In [5]:
del model

In [6]:
class img_dataset(torch.utils.data.Dataset):
    def __init__(self, imgs):
        self.imgs = imgs

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.to_list()
        
        inp = torch.Tensor(self.imgs[idx])
        inp = (inp / (255 / 2)) - 1
        inp = inp.permute((2, 0, 1))
        
        return inp    

In [22]:
%%time

noisy_cap = cv2.VideoCapture(noisy_path)
fps = noisy_cap.get(cv2.CAP_PROP_FPS)
width = int(noisy_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(noisy_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
out = cv2.VideoWriter(pred_save_path, fourcc, fps, (width, height))
    
frame_list = []

while noisy_cap.isOpened():
    ret, frame = noisy_cap.read()
    
    if ret == False:
        break
        
    frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
noisy_cap.release()
img_dataloader = torch.utils.data.DataLoader(img_dataset(frame_list), 
                                             batch_size = 256,
                                             shuffle = False,
                                            num_workers = 1)

# model.eval()
model_trt.eval()

preds_list = []

for ip in tqdm.tqdm(img_dataloader, total = len(img_dataloader)):    
    with torch.no_grad():
        
        ip = ip.to(device)
        preds = model_trt(ip).detach().cpu()

    preds_list.extend(list(preds))
    
for pred in preds_list:
    pred = ((pred + 1) / 2).permute(1, 2, 0)
    pred = (pred.numpy() * 255).astype('uint8')
    
    out.write(cv2.cvtColor(pred, cv2.COLOR_RGB2BGR))

torch.cuda.empty_cache()

100%|██████████| 6/6 [00:04<00:00,  1.35it/s]


CPU times: user 14.9 s, sys: 559 ms, total: 15.5 s
Wall time: 6.48 s
